# 1. Frame the Problem
Create a model that can be used in order to play a game of tetris for as long as possible.

# 2. Get the Data
We have been provided with the already coded game of tetris which will be used to create a model.

In [1]:
import numpy as np
import pandas as pd
import operator
from copy import copy, deepcopy
import random
from piece import BODIES, Piece
from board import Board
from myalgo import myai
from game import Game
from genetic_controller import compute_fitness

pygame 2.5.2 (SDL 2.28.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
population = 50
gens = 15
trials = 5
num_elite = 5
survival_rate=.2
logging_file='training4.csv'
headers = ['avg_fit','avg_gene', 'top_fit', 'top_gene', 'elite_fit', 'elite_gene']
pop = [myai(num_features = 3) for x in range(population)]

In [3]:
def playtetris(ai):
    game = Game(mode ='myai')
    pieces, cleared = game.run_no_visual()
    return pieces

In [4]:
def cross(a1, a2):
    new_genotype = []
    a1_prop = a1.fit_rel / a2.fit_rel
    for i in range(len(a1.genotype)):
        rand = random.uniform(0, 1)
        if rand > a1_prop:
            new_genotype.append(a1.genotype[i])
        else:
            new_genotype.append(a2.genotype[i])

    return myai(genotype=np.array(new_genotype), aggregate='lin', mutate=True)

In [6]:
for gen in range(gens):

        total_fitness = 0
        top_agent = 0
        gene = np.zeros(3)

        for n in range(population):
            print(f"Agent: {n}/{population}")
            agent = pop[n]
            agent.fit_score = compute_fitness(agent, num_trials=trials)
            total_fitness += agent.fit_score 
            #try:
            print(agent.genotype)
            print(gene)
            gene+=agent.genotype


        for agent in pop:
            agent.fit_rel = agent.fit_score / total_fitness

        next_gen = []

        sorted_pop = sorted(pop, reverse=True)

        elite_fit_score = 0
        elite_genes = np.zeros(3)
        top_agent=sorted_pop[0]

        for i in range(num_elite):
            elite_fit_score +=sorted_pop[i].fit_score
            elite_genes += sorted_pop[i].genotype
            next_gen.append(myai(genotype=sorted_pop[i].genotype, mutate=False))

       
        num_parents = round(population * survival_rate)
        parents = sorted_pop[:num_parents]

        
        for _ in range(population-(num_elite)):
            
            parents = random.sample(parents, 2)
            next_gen.append(cross(parents[0], parents[1]))


        avg_fit = (total_fitness/population)
        avg_gene = (gene/population)
        top_fit = (top_agent.fit_score)
        top_gene = (top_agent.genotype)
        elite_fit = (elite_fit_score/num_elite)
        elite_gene = (elite_genes/num_elite)

        data = [[avg_fit, avg_gene, top_fit, top_gene, elite_fit, elite_gene]]
        df = pd.DataFrame(data, columns=headers)
        df.to_csv(f'data/{logging_file}.csv', mode='a', index=False, header=False)


        print(f'\nEpoch {gens}: \n    total fitness: {total_fitness/population}\n    best agent: {top_agent.fit_score}\n')

        pop = next_gen

Agent: 0/50
84 18
    Trial: 0/5
62 8
    Trial: 1/5
86 19
    Trial: 2/5
86 18
    Trial: 3/5
60 9
    Trial: 4/5
[ 0.95599145 -0.90277841 -0.32021611]
[0. 0. 0.]
Agent: 1/50
11 0
    Trial: 0/5
10 0
    Trial: 1/5
10 0
    Trial: 2/5
9 0
    Trial: 3/5
9 0
    Trial: 4/5
[0.44970018 0.00224717 0.64180157]
[ 0.95599145 -0.90277841 -0.32021611]
Agent: 2/50
66 10
    Trial: 0/5
70 11
    Trial: 1/5
77 15
    Trial: 2/5
68 11
    Trial: 3/5
50 3
    Trial: 4/5
[ 0.53512214 -0.3355751  -0.37658077]
[ 1.40569162 -0.90053124  0.32158547]
Agent: 3/50
55 8
    Trial: 0/5
85 18
    Trial: 1/5
59 7
    Trial: 2/5
57 7
    Trial: 3/5
68 12
    Trial: 4/5
[ 0.42242516  0.81761098 -0.94712509]
[ 1.94081377 -1.23610633 -0.0549953 ]
Agent: 4/50
7 0
    Trial: 0/5
7 0
    Trial: 1/5
7 0
    Trial: 2/5
8 0
    Trial: 3/5
8 0
    Trial: 4/5
[-0.43576772 -0.67031905  0.16695372]
[ 2.36323893 -0.41849536 -1.00212039]
Agent: 5/50
12 1
    Trial: 0/5
11 0
    Trial: 1/5
10 0
    Trial: 2/5
10 0
    Trial: 

In [8]:
%run main.py myai

90 20
